In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
config = {
    "EPOCH": 300,
    "BATCH_SIZE": 50,
    "VALIDATION_BATCH_SIZE": 500,
    "DEVICE": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "NUM_STEPS": 1000,
    "AUGMENTATION_PROB": 0.5,
}

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 5)
        self.conv2 = nn.Conv2d(64, 64, 5)
        # size = 16 // 4 - 3
        self.fc1 = nn.Linear(64 * 5 * 5, 384)
        self.fc2 = nn.Linear(384, 192)
        self.fc3 = nn.Linear(192, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # x = self.pool(F.relu(self.conv1(x)))
        # x = self.pool(F.relu(self.conv2(x)))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
net = net.to(config["DEVICE"])

In [ ]:
random_transform = transforms.Compose(
    [
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(1),
        transforms.ColorJitter(0.9, 0.9),
    ]
)

# test_transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262]),
# ])

train_set = torchvision.datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.RandomApply([random_transform], config["AUGMENTATION_PROB"]),
            transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262]),
        ]
    ),
)

train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=config["BATCH_SIZE"], shuffle=True, num_workers=2
)

test_set = torchvision.datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262]),
        ]
    ),
)

test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=config["VALIDATION_BATCH_SIZE"], shuffle=False, num_workers=2
)

In [ ]:
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

criterion = nn.CrossEntropyLoss(reduction='sum')
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=4e-4)

scheduler = ReduceLROnPlateau(optimizer, "min", factor=0.5, min_lr=1e-6, verbose=True)

In [ ]:
def compute_loss_accuracy(net, dataloader):
    correct = 0

    with torch.no_grad():
        loss, n = 0, 0
        
        for data in dataloader:
            images, labels = data
            images = images.to(config["DEVICE"])
            labels = labels.to(config["DEVICE"])

            outputs = net(images)
            loss += criterion(outputs, labels)
            n += labels.size(0)
            
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()

    return loss / n, 100 * correct / n

In [ ]:
from tqdm.notebook import tqdm

# running_loss = 0.0
for j in tqdm(range(config["EPOCH"])):
    epoch_loss, n = 0, 0

    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.to(config["DEVICE"])
        labels = labels.to(config["DEVICE"])

        optimizer.zero_grad()

        outputs = net(inputs)
        train_loss = criterion(outputs, labels)

        epoch_loss += train_loss
        n += labels.size(0)

        train_loss = train_loss / labels.size(0)
        train_loss.backward()
        optimizer.step()

    # compute the average training loss during this epoch
    epoch_loss = epoch_loss / n

    test_loss, test_acc = compute_loss_accuracy(net, test_loader)

    scheduler.step(test_loss)
    print(f"Epoch {j + 1:3d}: train_loss={epoch_loss:.4f}, test_loss={test_loss:.4f}, test_acc={test_acc:.2f}")

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(config["DEVICE"])
        labels = labels.to(config["DEVICE"])

        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(
    f"Accuracy on {total} images = {100 * correct // total} % with {correct} images classified"
)